In [2]:
import random
from huggingface_hub import hf_api, hf_hub_download

# Paramètres du dataset ClimSim
REPO_ID = "LEAP/ClimSim_low-res"
# Le dossier 'test' contient normalement les dernières années (9 et 10)
FOLDER_PATH = "test" 

def download_random_samples(n=3):
    api = hf_api.HfApi()
    
    # 1. Lister tous les fichiers du dossier test
    files = api.list_repo_files(repo_id=REPO_ID, repo_type="dataset")
    test_files = [f for f in files if f.startswith(FOLDER_PATH) and f.endswith(".nc")]
    
    if not test_files:
        print("Aucun fichier trouvé. Vérifiez le chemin sur Hugging Face.")
        return

    # 2. Sélectionner au hasard
    selected_files = random.sample(test_files, min(n, len(test_files)))
    
    print(f"Téléchargement de {len(selected_files)} échantillons depuis l'année de test...")
    
    local_paths = []
    for file_path in selected_files:
        path = hf_hub_download(
            repo_id=REPO_ID, 
            filename=file_path, 
            repo_type="dataset"
        )
        local_paths.append(path)
        print(f"Téléchargé : {file_path}")
        
    return local_paths

# Exécution
samples = download_random_samples(3)

KeyboardInterrupt: 